#### https://towardsdatascience.com/classification-with-random-forests-in-python-29b8381680ed

In [1]:
import pandas as pd

In [9]:
pd.set_option('display.max_columns', 50)

In [169]:
pd.set_option('display.max_rows', 8500)

In [20]:
df = pd.read_csv('Data/mushrooms.csv')

In [6]:
print(df.shape)
print(df.columns)

(8124, 23)
Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')


#### Attribute Information: (classes: edible=e, poisonous=p)

cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s

cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s

cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y

bruises: bruises=t,no=f

odor: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s

gill-attachment: attached=a,descending=d,free=f,notched=n

gill-spacing: close=c,crowded=w,distant=d

gill-size: broad=b,narrow=n

gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y

stalk-shape: enlarging=e,tapering=t

stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?

stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s

stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s

stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y

stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y

veil-type: partial=p,universal=u

veil-color: brown=n,orange=o,white=w,yellow=y

ring-number: none=n,one=o,two=t

ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z

spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y

population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y

habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

In [10]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


In [14]:
df['cap-color'].unique()

array(['n', 'y', 'w', 'g', 'e', 'p', 'b', 'u', 'c', 'r'], dtype=object)

In [27]:
df_cat = pd.DataFrame()
for i in list(df.columns):
    df_cat['{}_cat'.format(i)] = df[i].astype('category').copy()
    df_cat['{}_cat'.format(i)] = df_cat['{}_cat'.format(i)].cat.codes

In [29]:
df_cat.dtypes

class_cat                       int8
cap-shape_cat                   int8
cap-surface_cat                 int8
cap-color_cat                   int8
bruises_cat                     int8
odor_cat                        int8
gill-attachment_cat             int8
gill-spacing_cat                int8
gill-size_cat                   int8
gill-color_cat                  int8
stalk-shape_cat                 int8
stalk-root_cat                  int8
stalk-surface-above-ring_cat    int8
stalk-surface-below-ring_cat    int8
stalk-color-above-ring_cat      int8
stalk-color-below-ring_cat      int8
veil-type_cat                   int8
veil-color_cat                  int8
ring-number_cat                 int8
ring-type_cat                   int8
spore-print-color_cat           int8
population_cat                  int8
habitat_cat                     int8
dtype: object

In [125]:
y = np.array(df_cat['class_cat'])
X = np.array(df_cat.drop('class_cat', axis = 1))

In [148]:
from sklearn.ensemble import RandomForestClassifier

In [255]:
from sklearn.model_selection import KFold
#kf = KFold(n_splits=5, random_state=42, shuffle=True)
kf = KFold(n_splits=5, random_state=None)

In [256]:
print(kf)

KFold(n_splits=5, random_state=None, shuffle=False)


In [257]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(train_index, test_index)
    print(len(train_index), len(test_index))
    print('*********')

[1625 1626 1627 ... 8121 8122 8123] [   0    1    2 ... 1622 1623 1624]
6499 1625
*********
[   0    1    2 ... 8121 8122 8123] [1625 1626 1627 ... 3247 3248 3249]
6499 1625
*********
[   0    1    2 ... 8121 8122 8123] [3250 3251 3252 ... 4872 4873 4874]
6499 1625
*********
[   0    1    2 ... 8121 8122 8123] [4875 4876 4877 ... 6497 6498 6499]
6499 1625
*********
[   0    1    2 ... 6497 6498 6499] [6500 6501 6502 ... 8121 8122 8123]
6500 1624
*********


In [264]:
results = []
from sklearn.metrics import f1_score
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = RandomForestClassifier(n_estimators=1, random_state=24)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(conf_matrix)
    results.append(f1_score(y_test, y_pred))

[[1273  159]
 [   0  193]]
[[1440    0]
 [   0  185]]
[[ 553    0]
 [   0 1072]]
[[ 264   12]
 [  59 1290]]
[[ 460   47]
 [   8 1109]]


In [265]:
import numpy as np
print('Accuracy:', round(np.mean(results), 5), '%.')

Accuracy: 0.93146 %.


In [260]:
results

[1.0, 1.0, 1.0, 0.996279761904762, 0.9933065595716197]

In [245]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[2123    0]
 [   0 1939]]


In [246]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
classification_report(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2123
           1       1.00      1.00      1.00      1939

    accuracy                           1.00      4062
   macro avg       1.00      1.00      1.00      4062
weighted avg       1.00      1.00      1.00      4062



'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00      2123\n           1       1.00      1.00      1.00      1939\n\n    accuracy                           1.00      4062\n   macro avg       1.00      1.00      1.00      4062\nweighted avg       1.00      1.00      1.00      4062\n'

In [247]:
# Precision is the ability of a classifier not to label an instance positive that is actually negative. TP/(TP + FP)
Precision = 1572/(1572+530)
# Recall is the ability of a classifier to find all positive instances. TP/(TP+FN)
Recall = 1572/(1572+551)
# The F1 score is a weighted harmonic mean of precision and recall such that the best score is 1.0 and the worst is 0.0. 
# Generally speaking, F1 scores are lower than accuracy measures as they embed precision and recall into their computation. 
# As a rule of thumb, the weighted average of F1 should be used to compare classifier models, not global accuracy.
f1_score = 2*(Recall * Precision) / (Recall + Precision)
print('Precision: {}, Recall: {}, F1_Score: {}'.format(Precision, Recall, f1_score))

Precision: 0.7478591817316841, Recall: 0.7404616109279322, F1_Score: 0.7441420118343195


In [248]:
2*(Recall * Precision) / (Recall + Precision)

0.7441420118343195